In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
from PIL import Image
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import cv2
import os
import glob
import pickle

Using TensorFlow backend.


In [2]:
filename = "Flickr8k.token.txt"
file = open(filename, 'r')
doc = file.read()
file.close()
count = 0
for line in doc.split('\n'):
    count = count + 1
print(count)
print(doc[:400])

40461
1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg#3	A little girl climbing the stairs to her playhouse .
1000268201_693b08cb0e.jpg#4	A little girl in a pink dress going into a wood


In [3]:
def load_descriptions(doc):
    mapping = dict()
    for line in doc.split('\n'):
        tokens = line.split()
        if len(line) < 2:
            continue
        image_id = tokens[0]
        image_desc = tokens[1:]
        image_desc = ' '.join(image_desc)
        image_id = image_id.split('.')[0]
        if image_id in mapping:
            mapping[image_id].append(image_desc)
        if image_id not in mapping:
            mapping[image_id] = list()
            mapping[image_id].append(image_desc)
    return mapping
descriptions = load_descriptions(doc)
print('loaded=', len(descriptions))

loaded= 8092


In [4]:
#print(descriptions)
print(list(descriptions.keys())[:5])

['1000268201_693b08cb0e', '1001773457_577c3a7d70', '1002674143_1b742ab4b8', '1003163366_44323f5815', '1007129816_e794419615']


In [5]:
print(len(descriptions))
print(descriptions['1000268201_693b08cb0e'])
print(descriptions['3712923460_1b20ebb131'])

8092
['A child in a pink dress is climbing up a set of stairs in an entry way .', 'A girl going into a wooden building .', 'A little girl climbing into a wooden playhouse .', 'A little girl climbing the stairs to her playhouse .', 'A little girl in a pink dress going into a wooden cabin .']
['a bunch of people in camo pants run .', 'A group of men and woman wearing army pants and a white shirt are running .', 'A group of men in camouflage pants and white t-shirts are running in a drill yard .', 'A group of people in army pants running .', 'A group of people wearing army clothes run together down the street .']


In [6]:
table = str.maketrans('', '', string.punctuation)
for key, desc_list in descriptions.items():
    #print(key, desc_list)
    for i in range(len(desc_list)):
        #print('1',desc_list[i])
        desc = desc_list[i]
        #print('2',desc)
        desc = desc.split()
        #print('3',desc)
        desc = [word.lower() for word in desc]
        #print('4',desc)
        desc = [w.translate(table) for w in desc]
        #print('5',desc)
        desc = [word for word in desc if (len(word) > 1)]
        #print('6',desc)
        desc = [word for word in desc if word.isalpha()]
        #print('7',desc)
        desc_list[i] = ' '.join(desc)
        #print('8',desc_list)
#print(desc_list)
#print(len(descriptions))
print(len(descriptions.keys()))

8092


In [7]:
print(descriptions['3712923460_1b20ebb131'])

['bunch of people in camo pants run', 'group of men and woman wearing army pants and white shirt are running', 'group of men in camouflage pants and white tshirts are running in drill yard', 'group of people in army pants running', 'group of people wearing army clothes run together down the street']


In [8]:
vocabulary = set()
for key in descriptions.keys():
    [vocabulary.update(d.split()) for d in descriptions[key]]
print('size = ', len(vocabulary))
print(len(descriptions))
print(len(descriptions.keys()))

size =  8763
8092
8092


In [9]:
train_captions = []
for key,val in descriptions.items():
    for captions in val:
        train_captions.append(captions)

wordcount_max = 10
word_count = {}
nsents = 0
for sent in train_captions:
    nsents = nsents + 1
    for w in sent.split():
        word_count[w] = word_count.get(w, 0) + 1
vocab = [w for w in word_count if word_count[w] > wordcount_max]
print('words = ', len(vocab))


words =  1839


In [10]:
print(vocab)

['child', 'in', 'pink', 'dress', 'is', 'climbing', 'up', 'set', 'of', 'stairs', 'an', 'way', 'girl', 'going', 'into', 'wooden', 'building', 'little', 'the', 'to', 'her', 'black', 'dog', 'and', 'spotted', 'are', 'fighting', 'tricolored', 'playing', 'with', 'each', 'other', 'on', 'road', 'white', 'brown', 'spots', 'staring', 'at', 'street', 'two', 'dogs', 'different', 'looking', 'pavement', 'moving', 'toward', 'covered', 'paint', 'sits', 'front', 'painted', 'rainbow', 'hands', 'bowl', 'sitting', 'large', 'small', 'grass', 'plays', 'it', 'there', 'pigtails', 'painting', 'young', 'outside', 'man', 'lays', 'bench', 'while', 'his', 'by', 'him', 'which', 'also', 'tied', 'sleeping', 'next', 'shirtless', 'lies', 'park', 'laying', 'holding', 'leash', 'ground', 'orange', 'hat', 'something', 'wears', 'glasses', 'wearing', 'beer', 'can', 'ears', 'rope', 'net', 'red', 'climbs', 'bridge', 'onto', 'ropes', 'playground', 'running', 'grassy', 'garden', 'surrounded', 'fence', 'through', 'terrier', 'green

In [11]:
############### already done. Dont execute again please #######################
f = open("vocabulary.txt", "a+")
for i in range(len(vocab)):
    f.write(vocab[i])
    f.write("\n")
f.close()

In [12]:
file = open('Flickr_8k.trainImages.txt')
doc = file.read()
train_images = list()
for word in doc.split('\n'):
    image_identifier = word.split('.')[0]
    train_images.append(image_identifier)
print(len(train_images))

6001


In [13]:
train_images

['2513260012_03d33305cf',
 '2903617548_d3e38d7f88',
 '3338291921_fe7ae0c8f8',
 '488416045_1c6d903fe0',
 '2644326817_8f45080b87',
 '218342358_1755a9cce1',
 '2501968935_02f2cd8079',
 '2699342860_5288e203ea',
 '2638369467_8fc251595b',
 '2926786902_815a99a154',
 '2851304910_b5721199bc',
 '3423802527_94bd2b23b0',
 '3356369156_074750c6cc',
 '2294598473_40637b5c04',
 '1191338263_a4fa073154',
 '2380765956_6313d8cae3',
 '3197891333_b1b0fd1702',
 '3119887967_271a097464',
 '2276499757_b44dc6f8ce',
 '2506892928_7e79bec613',
 '2187222896_c206d63396',
 '2826769554_85c90864c9',
 '3097196395_ec06075389',
 '3603116579_4a28a932e2',
 '3339263085_6db9fd0981',
 '2532262109_87429a2cae',
 '2076906555_c20dc082db',
 '2502007071_82a8c639cf',
 '3113769557_9edbb8275c',
 '3325974730_3ee192e4ff',
 '1655781989_b15ab4cbff',
 '1662261486_db967930de',
 '2410562803_56ec09f41c',
 '2469498117_b4543e1460',
 '69710415_5c2bfb1058',
 '3414734842_beb543f400',
 '3006217970_90b42e6b27',
 '2192411521_9c7e488c5e',
 '3535879138_928

In [14]:
train_descriptions = dict()
#train_descriptions_intermediate = list()
for line in str(descriptions).split('\n'):
    for image_id in descriptions:
        train_descriptions_intermediate = list()
        for i in range(len(descriptions[image_id])):
            train_descriptions_intermediate.append('start_sequence ' + ''.join(descriptions[image_id][i]) + ' end_sequence')
        train_descriptions[image_id] = train_descriptions_intermediate

In [15]:
train_descriptions

{'1000268201_693b08cb0e': ['start_sequence child in pink dress is climbing up set of stairs in an entry way end_sequence',
  'start_sequence girl going into wooden building end_sequence',
  'start_sequence little girl climbing into wooden playhouse end_sequence',
  'start_sequence little girl climbing the stairs to her playhouse end_sequence',
  'start_sequence little girl in pink dress going into wooden cabin end_sequence'],
 '1001773457_577c3a7d70': ['start_sequence black dog and spotted dog are fighting end_sequence',
  'start_sequence black dog and tricolored dog playing with each other on the road end_sequence',
  'start_sequence black dog and white dog with brown spots are staring at each other in the street end_sequence',
  'start_sequence two dogs of different breeds looking at each other on the road end_sequence',
  'start_sequence two dogs on pavement moving toward each other end_sequence'],
 '1002674143_1b742ab4b8': ['start_sequence little girl covered in paint sits in front

In [16]:
############### already done. Dont execute again please #######################
f = open("descriptions.txt", "w+")
for key,value in train_descriptions.items():
    f.write(key)
    f.write("\t")
    f.write(str(value))
    f.write("\n")
f.close()

In [17]:
print(train_descriptions['3712923460_1b20ebb131'])
print(train_descriptions['2279980395_989d48ae72'])
print(len(train_descriptions))
#print(train_descriptions.keys())

['start_sequence bunch of people in camo pants run end_sequence', 'start_sequence group of men and woman wearing army pants and white shirt are running end_sequence', 'start_sequence group of men in camouflage pants and white tshirts are running in drill yard end_sequence', 'start_sequence group of people in army pants running end_sequence', 'start_sequence group of people wearing army clothes run together down the street end_sequence']
['start_sequence girl is buried under several playground balls end_sequence', 'start_sequence laughing baby is surrounded by balls of various colors end_sequence', 'start_sequence smiling child is surrounded by colored balls end_sequence', 'start_sequence little girl lying in many colored plastic balls end_sequence', 'start_sequence the child is covered with colorful balls end_sequence']
8092


In [18]:
############### already done. Dont execute again please #######################
model = VGG16(weights = 'imagenet')

553467904/553467096 [==============================] - 1921s 3us/step


In [21]:
############### already done. Dont execute again please #######################

#model.add(Dense(2048))

model_new = Model(model.input, model.layers[-2].output)


In [22]:
############### already done. Dont execute again please #######################
model_new.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [23]:
images = 'Flicker8k_Dataset/'

img = glob.glob(images + '*.jpg')

In [24]:

train_images_file = 'Flickr_8k.trainImages.txt'

train_images = set(open(train_images_file, 'r').read().strip().split('\n'))

train_img = []

for i in img: 
    if i[len(images):] in train_images:
        train_img.append(i)

In [25]:
test_images_file = 'Flickr_8k.testImages.txt'

test_images = set(open(test_images_file, 'r').read().strip().split('\n'))

test_img = []

for i in img:
    if i[len(images):] in test_images:
        test_img.append(i)

In [26]:
############### already done. Dont execute again please #######################
#path = glob.glob('Flicker8k_Dataset/*.jpg')
#images = list()
#count = 0
def preprocess(image_path):    
    img = image.load_img(image_path)#, target_size=(299, 299))
    #n = cv2.imread(image_path)
    x = image.img_to_array(img)
    x = np.resize(x, (224, 224, 3))
    x = np.expand_dims(x, axis = 0)
    #y = preprocess_input(x)
    return x

In [27]:
############### already done. Dont execute again please #######################
def image_to_vector(img):
    img = preprocess(img)
    print(img.shape)
    x = model_new.predict(img)
    feature_vector_2048 = np.reshape(x, x.shape[1])
    return feature_vector_2048 

In [28]:
############### already done. Dont execute again please #######################
fea_vec_2048 = {}
count = 0
for img in train_img:
    if count <= 10:
        fea_vec_2048[img[len(images):]] = image_to_vector(img)
    #fea_vec_2048[img] = image_to_vector(img)
        count += 1
    else:
        break
    

(1, 224, 224, 3)


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node block1_conv1/convolution (defined at /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_keras_scratch_graph_637]

Function call stack:
keras_scratch_graph


In [ ]:
############### already done. Dont execute again please #######################
with open("encoded_train_images.pkl", "wb") as encoded_pickle:
    pickle.dump(fea_vec_2048, encoded_pickle)

In [ ]:
encoded_train_images = []
with open("encoded_train_images.pkl",'rb') as rfp: 
    while 1:
        try:
            encoded_train_images.append(pickle.load(rfp))
        except EOFError:
            break

In [ ]:
encoded_train_images.size

In [ ]:
train_images[0]


In [ ]:
"""
encoded_img ={}
for i in range(len(train_images)):
    #print(i)
    encoded_img[train_images[i]] = encoded_train_images[2048 * i: (2048 * i) + 2048 ]
"""

In [ ]:
encoded_img['1191338263_a4fa073154']
#len(encoded_img)

In [ ]:
f = open("vocabulary1.txt", "r")
if f.mode == "r":
    vocabfromfile = f.readlines()
print(len(vocabfromfile))
f.close()

In [ ]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocabfromfile:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [ ]:
print(ix)
print(len(ixtoword))
vocab_size = len(ixtoword) + 1

In [ ]:
f = open("descriptions.txt", "r")
if f.mode == "r":
    descriptions = f.readlines()
print(len(descriptions))
f.close()


In [ ]:
def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
    lines = to_lines(descriptions)
    return max(len(d.split()) for d in lines)

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

In [ ]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key]
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [ ]:
######################### already done this. dont do this again please ################
#glove_dir = 'glove.6B'
embeddings_index = {} # empty dictionary
f = open('glove.6B.200d.txt', encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embedding_dim = 200

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

In [ ]:
############ lstm model #########################3

inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [ ]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.optimizer.learning_rate = 0.0001
epochs = 10
number_pics_per_batch = 6
steps = len(train_descriptions)//number_pics_per_batch

In [ ]:
for i in range(epochs):
    generator = data_generator(train_descriptions, encoded_train_images, wordtoix, max_length, number_pics_per_batch)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)

In [ ]:
model.save("flickr/Flicker8k_Datset/model1.h5")